In [1]:
import socket
import pickle

# Create a TCP/IP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the IP address and port
server_socket.bind(('0.0.0.0', 12345))  # Listen on all available interfaces, port 12345

# Start listening for incoming connections
server_socket.listen(1)
print('Waiting for a connection...')

# Wait for a connection
client_socket, client_address = server_socket.accept()
print(f'Connection from {client_address}')

# Receive data
data = client_socket.recv(1024)  # Receive up to 1024 bytes of data
integers = pickle.loads(data) 
print(f'Received data: {integers}')

# Send a response (optional)
client_socket.sendall(b'Data received')

# Close the connection
client_socket.close()


Waiting for a connection...
Connection from ('10.21.236.238', 58274)
Received data: [1, 2, 3, 4, 5]


In [2]:
from pynq import Overlay, allocate
import numpy as np

overlay = Overlay("maxi.bit")

# Get the DMA instance
dma = overlay.axi_dma_0

# Number of integers to send
num_integers = len(integers)

# Allocate memory for input and output buffers
input_buffer = allocate(shape=(num_integers,), dtype=np.uint32)
output_buffer = allocate(shape=(1,), dtype=np.uint32)  # Single output for max value

# Fill the input buffer with some integers to test max finder
input_data = np.array(integers, dtype=np.uint32)
input_buffer[:] = input_data

# Initialize the output buffer with zeros
output_buffer[:] = np.zeros(1, dtype=np.uint32)

print("Input Buffer: ", input_buffer)

# Start the DMA send (from PS to PL)
dma.sendchannel.transfer(input_buffer)

# Start the DMA receive (from PL to PS)
dma.recvchannel.transfer(output_buffer)

# Wait for both send and receive transfers to complete
dma.sendchannel.wait()
dma.recvchannel.wait()

# Print the maximum value received from the IP
print("Maximum Value Received from IP: ", output_buffer)


Input Buffer:  [1 2 3 4 5]
Maximum Value Received from IP:  [5]
